In [1]:
from keras.layers import TextVectorization
import re
import tensorflow.strings as tf_strings
import json
import string
from keras.models import load_model
from tensorflow import argmax
from keras.preprocessing.text import tokenizer_from_json
from keras.utils import pad_sequences
import numpy as np
import tensorflow as tf

In [2]:
def beam_search_decoder(data, k):
    sequences = [[list(), 0.0]]
    for row in data:
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score - tf.math.log(row[j]).numpy()]
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:k]
    return sequences

In [4]:
#load  French model
model = load_model('english_to_french_model')

#load Tokenizer
with open('english_tokenizer.json') as f:
    data = json.load(f)
    english_tokenizer = tokenizer_from_json(data)
    
with open('French_tokenizer.json') as f:
    data = json.load(f)
    french_tokenizer = tokenizer_from_json(data)
    

#load max length
with open('sequence_length.json') as f:
    max_length = json.load(f)
    
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

english_sentence = "new jersey is sometimes quiet during autumn , and it is snowy in april ."
english_sentence = english_sentence.lower()

english_sentence = english_sentence.replace(".", '')
english_sentence = english_sentence.replace("?", '')
english_sentence = english_sentence.replace("!", '')
english_sentence = english_sentence.replace(",", '')

english_sentence = english_tokenizer.texts_to_sequences([english_sentence])
english_sentence = pad(english_sentence, max_length)

english_sentence = english_sentence.reshape((-1,max_length))

french_sentence = model.predict(english_sentence)


1/1 [==============================] - 3s 3s/step


In [5]:
data = np.transpose(french_sentence, (1, 0, 2))[0]

beam_width = 3
decoded_sequences = beam_search_decoder(data, beam_width)
print("Decoded sequences:", decoded_sequences)

Decoded sequences: [[[35], 0.0], [[34], 20.797693252563477], [[4], 25.716947555541992]]


In [6]:
decoded_text = []
for i in range(3):
    decoded_text.extend(french_tokenizer.sequences_to_texts([decoded_sequences[i][0]]))

In [7]:
decoded_text

['new', 'jersey', 'les']